In [ ]:
import sys
from matplotlib import pyplot
from keras.datasets import cifar10
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.optimizers import SGD
from keras.regularizers import l2
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import pandas as pd

# load train and test dataset
def load_dataset():
	# load dataset
  # trainX=[]
  # trainY=[]
  # testX=[]
  # testY=[]
  (trainX, trainY), (testX, testY) = cifar10.load_data()
  # for item in range(5000):
  #   trainX.append(train_X[item])
  #   trainY.append(train_Y[item])
  #   testX.append(test_X[item])
  #   testY.append(test_Y[item])
	# one hot encode target values
  trainY = to_categorical(trainY)
  testY = to_categorical(testY)
  return trainX, trainY, testX, testY


# New Section

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
def prep_pixels(train, test):
	# convert from integers to floats
  print("prepping pixels")
  train_norm = np.array(train, dtype=np.float32)
  test_norm = np.array(test, dtype=np.float32)
	# normalize to range 0-1
  train_norm = train_norm / 255.0
  test_norm = test_norm / 255.0
  # return normalized images
  return train_norm, test_norm
 
# define cnn model
#The model consists of of 3 initial layers and uses a relu activation function
def define_model():
	model = Sequential()
	model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same',kernel_regularizer=l2(0.001), input_shape=(32, 32, 3)))
	model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same',kernel_regularizer=l2(0.001)))
	model.add(MaxPooling2D((2, 2)))
	model.add(Dropout(0.2))
  # 20% of the nodes are removed using a dropout function to help regularization
	model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same',kernel_regularizer=l2(0.001)))
	model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same',kernel_regularizer=l2(0.001)))
	model.add(MaxPooling2D((2, 2)))
	model.add(Dropout(0.2))
	model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same',kernel_regularizer=l2(0.001)))
	model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same',kernel_regularizer=l2(0.001)))
	model.add(MaxPooling2D((2, 2)))
	model.add(Dropout(0.2))
	model.add(Flatten()) #the layers a flattened out in this layer to pass into the next layer
	model.add(Dense(128, activation='relu', kernel_initializer='he_uniform',kernel_regularizer=l2(0.001)))
	model.add(Dropout(0.2))
	model.add(Dense(10, activation='softmax')) #using a softmax activation in the final layer to 
	# compile model
	opt = SGD(lr=0.001, momentum=0.9)
	model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
	return model
 
# plot diagnostic learning curves
def summarize(history):
	# plot loss
	pyplot.subplot(211)
	pyplot.title('Cross Entropy Loss')
	pyplot.plot(history.history['loss'], color='blue', label='train')
	pyplot.plot(history.history['val_loss'], color='orange', label='test')
	# plot accuracy
	pyplot.subplot(212)
	pyplot.title('Classification Accuracy')
	pyplot.plot(history.history['accuracy'], color='blue', label='train')
	pyplot.plot(history.history['val_accuracy'], color='orange', label='test')
	# save plot to file
	filename = sys.argv[0].split('/')[-1]
	pyplot.savefig(filename + '_plot.png')
	pyplot.close()
 
# run the test harness for evaluating a model
def run_test():
	trainX, trainY, testX, testY = load_dataset()
	trainX, testX = prep_pixels(trainX, testX)
	model = define_model()
	model.fit(trainX, trainY, epochs=100, batch_size=64, verbose=0)#fit the made model into the data
	model.save('final_model.h5')

run_test()

170500096/170498071 [==============================] - 2s 0us/step
prepping pixels


In [ ]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.models import load_model

def load_image(filename):  #preparing the uploaded image 
	img = load_img(filename, target_size=(32, 32))
	img = img_to_array(img)
	pyplot.imshow(img.reshape(32,32,3)/255,interpolation='nearest')
	img = img.reshape(1, 32, 32, 3)
	img = img.astype('float32')
	img = img / 255.0
	return img
 
def run_example(filename):
	img = load_image(filename)
	model = load_model('model.h5')
	result = model.predict_classes(img)
	return result[0]

#Get all Labels
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict
batches=unpickle("/content/batches.meta")
label_names=batches[b'label_names']

In [ ]:
test_image='/content/1.jfif'   #Change test image HERE to predict
label_names[run_example(test_image)]